https://github.com/tiru-patel/Medical_Text_Summarization_using_LLMs

https://github.com/abhimishra91/transformers-tutorials/blob/master/transformers_summarization_wandb.ipynb

https://github.com/mrdbourke/tensorflow-deep-learning/blob/main/05_transfer_learning_in_tensorflow_part_2_fine_tuning.ipynb

https://github.com/abhimishra91/transformers-tutorials/blob/master/transformers_summarization_wandb.ipynb





---



**Introduction**
In this tutorial we will be fine tuning a transformer model for Summarization Task. In this task a summary of a given article/document is generated when passed through a network. There are 2 types of summary generation mechanisms:

**Extractive Summary:** the network calculates the most important sentences from the article and gets them together to provide the most meaningful information from the article.

**Abstractive Summary:** The network creates new sentences to encapsulate maximum gist of the article and generates that as output. The sentences in the summary may or may not be contained in the article.
In this tutorial we will be generating Abstractive Summary.



---



https://github.com/JersonGB22/Summarization-TensorFlow/blob/main/models/Summarization_T5_XSum.ipynb

  The Text-To-Text Transfer Transformer (T5) model, specifically the Base version with 220 million parameters, available on Hugging Face 🤗, will be fine-tuned. The fine-tuning is performed using the Extreme Summarization (XSum) Dataset, which contains BBC news articles (mostly quite extensive) along with their corresponding concise single-sentence summaries.



---



https://github.com/aravindpai/How-to-build-own-text-summarizer-using-deep-learning/blob/master/How_to_build_own_text_summarizer_using_deep_learning.ipynb

In this notebook, we will build an abstractive based text summarizer using deep learning from the scratch in python using keras

I recommend you to go through the article over here to cover all the concepts which is required to build our own summarizer

In [1]:
from google.colab import drive
drive.mount('/content/drive')
datasets_path = '/content/drive/MyDrive/ml_datasets'

Mounted at /content/drive


In [2]:
# Install the Hugging Face datasets library
!pip install datasets -q

# Install the necessary library to evaluate the summary
!pip install rouge_score -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
  Preparing metadata (setup.py) ... done


In [3]:
import numpy as np
import re
import plotly.graph_objects as go
from google.colab import drive
from IPython.core.display import HTML
import textwrap
from tqdm.auto import tqdm
import pandas as pd
import os

import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM, AdamWeightDecay
from datasets import load_dataset, Dataset #, load_metric
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
model_name = "t5-base" ##Text-To-Text Transfer Transformer
max_context_length = 1024
max_target_length = 128
batch_size = 32
SEED = 123
epochs = 1  ##Recoomended higher but for faster execution using 1 epoch only for now
prefix = "summarize: " # The task prefix in T5 models is important for achieving better results

In [5]:
## Extreme Summarization (XSum) Dataset, which contains BBC news articles (mostly quite extensive) along with their corresponding concise single-sentence summaries.
(train_dataset, val_dataset) = load_dataset("xsum", split=["train", "validation"], trust_remote_code=True)

print(train_dataset)
print(val_dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/6.24k [00:00<?, ?B/s]

xsum.py:   0%|          | 0.00/5.76k [00:00<?, ?B/s]

(…)SUM-EMNLP18-Summary-Data-Original.tar.gz:   0%|          | 0.00/255M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 204045
})
Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 11332
})


In [6]:
# See an example of the training dataset
train_dataset[0]

{'document': 'The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.\nRepair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.\nTrains on the west coast mainline face disruption due to damage at the Lamington Viaduct.\nMany businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.\nFirst Minister Nicola Sturgeon visited the area to inspect the damage.\nThe waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.\nJeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.\nHowever, she said more preventative work could have been carried out to ensure the retaining wall did not fail.\n"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally apprecia

In [7]:
'''
The XSum dataset contains noise: some records are empty, have only one character,
or the document and summary do not correspond to each other or are incoherent.
To reduce this noise, records with 10 words or fewer in the document and records with 5 words or fewer in the summary are removed.
Additionally, hyperlinks, emails, and Twitter usernames are removed from the records.
Below, you can see some examples of incoherent records:
'''

print(train_dataset[964], "\n")
print(train_dataset[7685], "\n")
print(train_dataset[54003], "\n")

{'document': 'Amateur rocket enthusiasts have gathered in Scotland to mark 30 years of aiming for the skies.\nInternational Rocket Week 2016 traces its roots back to the first Scottish Rocket Weekend in 1986.\nRocketeers coming together in Scotland have a base camp near Paisley and launch to heights of up to 16,000 ft (4,876m) from a farm near Largs.', 'summary': '.', 'id': '37185211'} 

{'document': '', 'summary': "As part of the latest series of African Dream, businesswoman Mercy Kitomari - the brains behind Tanzanian organic ice cream company Nelwa's Gelato - shares her top 10 social media marketing tips for aspiring entrepreneurs:", 'id': '30280969'} 

{'document': "Here's my take.", 'summary': "It's much better than it used to be, but female leadership in the United States - in politics and business - has still a long way to go to achieve parity.", 'id': '39213019'} 



In [8]:
def preprocessing(text):
  # Remove hyperlinks
  text = re.sub(r"https?://[^\s\n\r]+|www\.[^\s\n\r]+", "", text)
  # Remove Twitter usernames and email addresses
  text = re.sub(r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}\b", "", text)
  text = re.sub(r"@\w+", "", text)
  return text

preprocessing('ak@gmail.com has opend an account on www.google.com')

' has opend an account on '

In [9]:
def preprocess_dataset(dataset):
  contexts, targets = [], []

  for sample in tqdm(dataset):
    context = preprocessing(sample["document"])
    target = preprocessing(sample["summary"])

    if (len(context.split()) > 10) and (len(target.split()) > 5):
      contexts.append(prefix + context)   ## Notice we are appending prefix 'summarize' to the context
      targets.append(target)

  new_dataset = Dataset.from_dict({"document": contexts, "summary": targets})
  return new_dataset

sample_doc = {
    'document': "Have you been affected by flooding in Dumfries and Galloway or the Borders? Tell us about your experience of the situation and how it was handled. Email us on selkirk.news@bbc.co.uk or dumfries@bbc.co.uk.",
    'summary': "If affected by something, tell experience at news@bbc.co.uk"
}
preprocess_dataset([sample_doc])
preprocess_dataset([sample_doc])[0]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

{'document': 'summarize: Have you been affected by flooding in Dumfries and Galloway or the Borders? Tell us about your experience of the situation and how it was handled. Email us on  or .',
 'summary': 'If affected by something, tell experience at '}

In [10]:
train_dataset = preprocess_dataset(train_dataset)
val_dataset = preprocess_dataset(val_dataset)

  0%|          | 0/204045 [00:00<?, ?it/s]

  0%|          | 0/11332 [00:00<?, ?it/s]

In [11]:
print(len(train_dataset), len(val_dataset))

((204045-len(train_dataset)) / 204045) * 100, ((11332-len(val_dataset)) / 11332) * 100
## It can be seen that the discarded records after preprocessing the datasets represent no more than 0.34% of each dataset.

203354 11295


(0.33865078781641306, 0.3265090010589481)

**Tokenize the Dataset**

For tokenizing the datasets, the AutoTokenizer from Hugging Face is used, allowing the creation of a tokenizer instance based solely on the model name. In this text summarization task, both document and summary texts are tokenized. The tokenizer generates input_ids (document tokens), attention_mask (a mask to ignore the padding token indices in input_ids), and labels (summary tokens). In the specific case of the T5 model, decoder_input_ids (the labels tokens shifted one position to the right with a padding token at the start) do not need to be passed during training, as the model generates them automatically from labels.

In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

print('name_or_path: ', tokenizer.name_or_path)
print('vocab_size: ', tokenizer.vocab_size)
print('model_max_length: ', tokenizer.model_max_length)
print('padding_side: ', tokenizer.padding_side)
print('truncation_side: ', tokenizer.truncation_side)

# tokenizer
# print(tokenizer.special_tokens)
# tokenizer.added_tokens_decoder

import itertools
dict(itertools.islice(tokenizer.vocab.items(), 10))

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

name_or_path:  t5-base
vocab_size:  32100
model_max_length:  1000000000000000019884624838656
padding_side:  right
truncation_side:  right


{'▁blanche': 21403,
 '▁mai': 187,
 '▁da': 836,
 '▁Google': 1163,
 '▁recital': 30731,
 'incoming': 19583,
 '▁comentarii': 27392,
 'hum': 4884,
 '▁factors': 2580,
 'cina': 11075}

In [13]:
# Example of tokenization
sample_doc = {
    'document': "Have you been affected by flooding in Dumfries and Galloway or the Borders? Tell us about your experience of the situation and how it was handled. Email us on selkirk.news@bbc.co.uk or dumfries@bbc.co.uk.",
    'summary': "If affected by something, tell experience at news@bbc.co.uk"
}
# id = 9
# example = train_dataset[id]
# encoding = tokenizer(example["document"], text_target=example["summary"])
encoding = tokenizer(sample_doc["document"], text_target=sample_doc["summary"])
print(encoding.keys())

input_ids = encoding["input_ids"]
labels = encoding["labels"]

dict_keys(['input_ids', 'attention_mask', 'labels'])


In [14]:
print('sample_doc: ', sample_doc)
print(f"Input Ids:\n{input_ids}\n")
print(f"Tokens:\n{tokenizer.convert_ids_to_tokens(input_ids)}\n")
print(f"Decoded Text without Special Tokens:\n{textwrap.fill(tokenizer.decode(input_ids, skip_special_tokens=True), width=150)}\n")
print(f"Attention Mask:\n{encoding['attention_mask']}\n\n")

sample_doc:  {'document': 'Have you been affected by flooding in Dumfries and Galloway or the Borders? Tell us about your experience of the situation and how it was handled. Email us on selkirk.news@bbc.co.uk or dumfries@bbc.co.uk.', 'summary': 'If affected by something, tell experience at news@bbc.co.uk'}
Input Ids:
[2114, 25, 118, 4161, 57, 18368, 16, 970, 51, 89, 2593, 11, 10987, 32, 1343, 42, 8, 17600, 7, 58, 8779, 178, 81, 39, 351, 13, 8, 1419, 11, 149, 34, 47, 10298, 5, 8601, 178, 30, 142, 40, 157, 12546, 5, 15808, 1741, 115, 115, 75, 5, 509, 5, 1598, 42, 146, 51, 89, 2593, 1741, 115, 115, 75, 5, 509, 5, 1598, 5, 1]

Tokens:
['▁Have', '▁you', '▁been', '▁affected', '▁by', '▁flooding', '▁in', '▁Du', 'm', 'f', 'ries', '▁and', '▁Gall', 'o', 'way', '▁or', '▁the', '▁Border', 's', '?', '▁Tell', '▁us', '▁about', '▁your', '▁experience', '▁of', '▁the', '▁situation', '▁and', '▁how', '▁it', '▁was', '▁handled', '.', '▁Email', '▁us', '▁on', '▁se', 'l', 'k', 'irk', '.', 'news', '@', 'b', 'b', '

In [15]:
print(f"Label:\n{labels}\n")
print(f"Tokens:\n{tokenizer.convert_ids_to_tokens(labels)}\n")
print(f"Decoded Text without Special Tokens:\n{textwrap.fill(tokenizer.decode(labels, skip_special_tokens=True), width=150)}")

Label:
[156, 4161, 57, 424, 6, 817, 351, 44, 1506, 1741, 115, 115, 75, 5, 509, 5, 1598, 1]

Tokens:
['▁If', '▁affected', '▁by', '▁something', ',', '▁tell', '▁experience', '▁at', '▁news', '@', 'b', 'b', 'c', '.', 'co', '.', 'uk', '</s>']

Decoded Text without Special Tokens:
If affected by something, tell experience at news@bbc.co.uk


In [16]:
def tokenize_dataset(dataset):
  contexts, targets = dataset["document"], dataset["summary"]

  inputs = tokenizer(
      contexts,
      padding="max_length",
      truncation=True,
      max_length=max_context_length
  )

  labels = tokenizer(
      text_target=targets,
      padding="max_length",
      truncation=True,
      max_length=max_target_length,
      return_tensors="np"
  )["input_ids"]

  labels[labels == tokenizer.pad_token_id] = -100
  inputs["labels"] = labels

  return inputs

In [17]:
train_dataset = train_dataset.map(tokenize_dataset, batched=True)
val_dataset = val_dataset.map(tokenize_dataset, batched=True)

Map:   0%|          | 0/203354 [00:00<?, ? examples/s]

Map:   0%|          | 0/11295 [00:00<?, ? examples/s]

**Creation and Compilation of the Model**

Due to the large size of the training dataset, the tokenized document sequences (1024 tokens) are also large, and the T5 Base model is relatively large with over 220 million parameters, it is necessary to fine-tune the model on the Google Colab TPU to significantly speed up training time. Therefore, as noted in tokenization, the sequences must have a fixed size. If they were dynamically padded to the longest sequence in the batch, the training would be slower than training on the Colab CPU.

In [18]:
### TPU setup
# try:
#   tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu="local")
#   tf.config.experimental_connect_to_cluster(tpu)
#   tf.tpu.experimental.initialize_tpu_system(tpu)
#   strategy = tf.distribute.TPUStrategy(tpu)
#   print("TPU initialized:")
#   print("All devices: ", tf.config.list_logical_devices("TPU"))
# except ValueError:
#   print("TPU not found")

In [19]:
def masked_accuracy(y_true, y_pred):
  y_pred = tf.cast(tf.argmax(y_pred, axis=-1), y_true.dtype)
  mask = tf.cast(y_true != -100, tf.float32)
  accuracy = tf.cast(y_true == y_pred, tf.float32)
  accuracy *= mask

  return tf.reduce_sum(accuracy) / tf.reduce_sum(mask)

In [20]:
tf.keras.backend.clear_session() #Clearing Keras memory
tf.random.set_seed(SEED) #For reproducibility

# with strategy.scope():
#   ...

## Download the T5 model
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_name)
optimizer = AdamWeightDecay(learning_rate=3e-4, weight_decay_rate=0.01)

model.compile(
    optimizer=optimizer,
    metrics=[masked_accuracy],
    steps_per_execution=400
)

model.summary()

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


Model: "tft5_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  24674304  
                                                                 
 encoder (TFT5MainLayer)     multiple                  109628544 
                                                                 
 decoder (TFT5MainLayer)     multiple                  137949312 
                                                                 
Total params: 222903552 (850.31 MB)
Trainable params: 222903552 (850.31 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


https://fabianofalcao.medium.com/metrics-for-evaluating-summarization-of-texts-performed-by-transformers-how-to-evaluate-the-b3ce68a309c3

https://neptune.ai/blog/keras-metrics

In [21]:
np.random.seed(SEED)
tf.random.set_seed(SEED)

train_dataset_tf = model.prepare_tf_dataset(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    drop_remainder=True
)

val_dataset_tf = model.prepare_tf_dataset(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    drop_remainder=True
)

In [22]:
# Viewing a training batch
for inputs, labels in train_dataset_tf.take(1):
  print(inputs["input_ids"], "\n")
  print(inputs["attention_mask"], "\n")
  print(labels)

tf.Tensor(
[[21603    10 17113 ...     0     0     0]
 [21603    10    37 ...     0     0     0]
 [21603    10 16453 ...     0     0     0]
 ...
 [21603    10 20264 ...     0     0     0]
 [21603    10    37 ...     0     0     0]
 [21603    10 27344 ...     0     0     0]], shape=(32, 1024), dtype=int64) 

tf.Tensor(
[[1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 ...
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]], shape=(32, 1024), dtype=int64) 

tf.Tensor(
[[   37  2734 13235 ...  -100  -100  -100]
 [ 2158   651  3038 ...  -100  -100  -100]
 [   37  2753    13 ...  -100  -100  -100]
 ...
 [   37   166  2553 ...  -100  -100  -100]
 [10256    31  2630 ...  -100  -100  -100]
 [24191     7    13 ...  -100  -100  -100]], shape=(32, 128), dtype=int64)


**Fine-Tuning the Model**

Even with the Colab TPU, training takes more than 3 hours, which is the maximum time provided by the free version of Colab. Therefore, it is necessary to use checkpoints to save the model and the optimizer after a certain number of epochs and continue training from where it left off.

In [23]:
checkpoint_folder = "/content/drive/MyDrive/checkpoints/Summarization_T5_XSum"
checkpoint = tf.train.Checkpoint(model=model, optimizer=optimizer)

In [ ]:
# pretrained = True #Change according to previous training

# if pretrained:
#   last_epoch = 4 #Put the last epoch of the pretrained model

#   # Restore the model and the optimizer from the latest checkpoint
#   checkpoint.restore(tf.train.latest_checkpoint(checkpoint_folder))
#   history = pd.read_csv(os.path.join(checkpoint_folder, f"history_epoch_{last_epoch}.csv"))

# else:
#   last_epoch = 0
#   os.makedirs(checkpoint_folder, exist_ok=True)
#   history = pd.DataFrame({"loss": [], "val_loss": [], "accuracy": [], "val_accuracy": []})

In [ ]:
# ## Callback to save checkpoints and training history
# class CheckpointCallback(tf.keras.callbacks.Callback):
#   def __init__(self, save_path, save_freq, history):
#     super().__init__()
#     self.save_path = save_path
#     self.save_freq = save_freq
#     self.history = history

#   def on_epoch_end(self, epoch, logs=None):
#     epoch_history = pd.DataFrame({k: [v] for k, v in logs.items()})
#     self.history = pd.concat([self.history, epoch_history], axis=0, ignore_index=True)
#     self.history.to_csv(os.path.join(self.save_path, f"history_epoch_{epoch+1}.csv"), index=False)

#     if (epoch+1) % self.save_freq == 0:
#       checkpoint.save(file_prefix=os.path.join(self.save_path, f"epoch_{epoch+1}"))

In [ ]:
# checkpoint_callback = CheckpointCallback(checkpoint_folder, save_freq=4, history=history)

# early_callback = tf.keras.callbacks.EarlyStopping(
#     monitor="val_masked_accuracy",
#     patience=1,
#     mode="max",
#     restore_best_weights=True
# )

In [ ]:
tf.random.set_seed(SEED)

history = model.fit(
    train_dataset_tf,
    epochs=epochs,
    validation_data=val_dataset_tf
    # callbacks=[checkpoint_callback, early_callback],
    # initial_epoch=last_epoch
)

## if TPU is not used, OOM error is being thrown

In [ ]:
trained_model_path = "/content/drive/MyDrive/models/Summarization_T5_XSum"
model.save_pretrained(trained_model_path)

**Model Evaluation**

Conventional **accuracy** (train_dataset: 71.99%, val_dataset: 61.61%) does not accurately reflect the model's true performance but serves as a useful approximation during training to gauge how well the model is performing in text summarization. For a more precise evaluation, the **ROUGE** metric will be used.

The Recall-Oriented Understudy for Gisting Evaluation (ROUGE) metric is a set of measures used to evaluate the quality of text translation and summarization. It includes several variants, such as rouge1, which measures the overlap of unigrams between the generated summary and the reference summary; rouge2, which evaluates the overlap of bigrams; rougeL, which considers the longest sequence of overlapping tokens; and rougeLsum, which computes ROUGE-L at the summary level. Additionally, these metrics can be assessed at different levels of granularity, such as low, mid, and high, representing various thresholds of overlap. Each component of the ROUGE metric is evaluated on a scale from 0 to 1, where 1 indicates an exact match between the predictions and the references.

In [ ]:
## With conventional Accuracy
print(model.evaluate(train_dataset_tf, verbose=0))
print(model.evaluate(val_dataset_tf, verbose=0))

## Also, per epoch how valiation and training loss and accuracy changes can be plotted

In [ ]:
metric = load_metric("rouge")
metric

In [ ]:
def prepare_for_evaluation(dataset, batch_size=batch_size):
  list_predictions, list_references = [], []

  for i in tqdm(range(0, len(dataset["document"]), batch_size)):
    batch_data = dataset[i: i+batch_size]

    predictions = model.generate(
        input_ids=tf.constant(batch_data["input_ids"]),
        attention_mask=tf.constant(batch_data["attention_mask"]),
        max_length=max_target_length
    )
    predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # It is necessary to separate sentences with '\n' to accurately calculate the ROUGE metric
    predictions = ["\n".join(nltk.sent_tokenize(prediction)) for prediction in predictions]
    references = ["\n".join(nltk.sent_tokenize(reference)) for reference in batch_data["summary"]]

    list_predictions += predictions
    list_references += references

  return list_predictions, list_references

predictions, references = prepare_for_evaluation(val_dataset, batch_size=batch_size)
result = metric.compute(predictions=predictions, references=references, use_stemmer=True)
result

In [ ]:
## Generating Summaries with the Trained Model


In [ ]:
# Function to generate summaries and display them in HTML format

def display_summary(text, reference=None):
  text = preprocessing(text)
  inputs = tokenizer(
      [prefix + text],
      truncation=True,
      max_length=max_context_length,
      return_tensors="tf"
  )

  prediction = model.generate(**inputs, max_length=max_target_length)[0]
  prediction = tokenizer.decode(prediction, skip_special_tokens=True)

  #Replace `\n` with `<br>` to make the line break visible in HTML format
  text = text.replace("\n", "<br>")
  content_html = f"""
  <b>Text:<br></b> {text}<br><br>
  <b>Summary:</b> {prediction}<br><br>
  """

  if reference is not None:
    reference = preprocessing(reference)

    result = metric.compute(
        predictions=["\n".join(nltk.sent_tokenize(prediction))],
        references=["\n".join(nltk.sent_tokenize(reference))],
        use_stemmer=True
    )
    result = {key: value.mid.fmeasure*100 for key, value in result.items()}

    content_html += f"<b>Reference:</b> {reference}<br><br>"
    for key, value in result.items():
      content_html += f"<b><span style='color: blue'>{key}:</span></b> {round(value, 2)}&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"

  display(HTML(content_html))

In [ ]:
text = """
Facebook has announced a new feature that allows users to create custom avatars for use in comments and messages.
The avatars can be personalized to look like the users themselves, with options for different hairstyles, outfits,
and accessories. This move is seen as a direct response to similar features offered by Snapchat and Apple.
"""

display_summary(text)